# ChromaDB Experiment Example

## Installations

In [1]:
# !pip install --quiet --force-reinstall prompttools

## Setup imports and API keys

We'll import the relevant `prompttools` modules to setup our experiment.

In [2]:
from prompttools.experiment import ChromaDBExperiment
import chromadb

## Run an experiment

One common use case is to compare two different embedding functions and how it may impact your document retrieval. We have can define what embedding functions we'd like to test here.

Note: If you previously haven't downloaded these embedding models. This may kick off downloads.

In [3]:
from chromadb.utils import embedding_functions


emb_fns = [
    embedding_functions.SentenceTransformerEmbeddingFunction(model_name="paraphrase-MiniLM-L3-v2"),
    embedding_functions.DefaultEmbeddingFunction(),
]  # default is "all-MiniLM-L6-v2"
emb_fns_names = ["paraphrase-MiniLM-L3-v2", "default"]

/Users/kevin/miniconda3/envs/prompttools/lib/python3.11/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/Users/kevin/miniconda3/envs/prompttools/lib/python3.11/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
/Users/kevin/miniconda3/envs/prompttools/lib/python3.11/site-packages/tensorflow/python/debug/cli/debugger_cli_common.py:19: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants


Next, we create our test inputs. In this case, we would like to create a new ChromaDB collection.

During the experiment, for each embedding function, a new ChromaDB collection will be temporarily created. The documents will be added into it. Then, we will query from it and examine the results.

You can test different chunking and pre-processing strategies by changing the input to the `"documents"` key of the dictionary. We will show a more complete example further below.

In [4]:
chroma_client = chromadb.Client()
# You can also create and use `chromadb.PersistentClient` or `chromadb.HttpClient`
TEST_COLLECTION_NAME = "TEMPORARY_COLLECTION"
try:
    chroma_client.delete_collection(TEST_COLLECTION_NAME)
except Exception:
    pass
collection_name = TEST_COLLECTION_NAME

use_existing_collection = False  # Specify that we want to create a collection during the experiment

# Documents that will be added into the database
# Based on your chunking and pre-processing steps, the `documents` will be different
add_to_collection_params = {
    "documents": ["This is a document", "This is another document", "This is the document."],
    "metadatas": [{"source": "my_source"}, {"source": "my_source"}, {"source": "my_source"}],
    "ids": ["id1", "id2", "id3"],
}

In [ ]:
# Our test queries
query_collection_params = {"query_texts": ["This is a query document", "This is a another query document"]}


# Set up the experiment
experiment = ChromaDBExperiment(
    chroma_client,
    collection_name,
    use_existing_collection,
    query_collection_params,
    emb_fns,
    emb_fns_names,
    add_to_collection_params,
)

We can then run the experiment to get results.

In [5]:
experiment.run()

/Users/kevin/miniconda3/envs/prompttools/lib/python3.11/site-packages/chromadb/utils/read_write_lock.py:29: DeprecationWarning: notifyAll() is deprecated, use notify_all() instead
  self._read_ready.notifyAll()


We can visualize the result. In this case, the result of the second query "This is a another query document" is different.

paraphrase-MiniLM-L3-v2: [id2, id3, id1]

default (all-MiniLM-L6-v2) : [id2, id1, id3]

In [6]:
experiment.visualize()

,query_texts,embed_fn,top doc ids,distances,documents,latency
0,This is a query document,paraphrase-MiniLM-L3-v2,"[id1, id3, id2]","[14.106966018676758, 14.294026374816895, 18.137874603271484]","[This is a document, This is the document., This is another document]",0.008508
1,This is a another query document,paraphrase-MiniLM-L3-v2,"[id2, id3, id1]","[13.375584602355957, 16.815608978271484, 16.913410186767578]","[This is another document, This is the document., This is a document]",0.006097
2,This is a query document,default,"[id1, id3, id2]","[0.7111212611198425, 0.8084275126457214, 1.010977029800415]","[This is a document, This is the document., This is another document]",0.020234
3,This is a another query document,default,"[id2, id1, id3]","[0.7673601508140564, 0.8709302544593811, 0.9072309732437134]","[This is another document, This is a document, This is the document.]",0.020889


## Evaluate the model response

To evaluate the results, we'll define an evaluation function. Sometimes, you know order of the most relevant document should be given a query, and you can compute the correlation between expected ranking and actual ranking.

Note: there is a built-in version of this function that you can import (scroll further below to see an example).

In [7]:
import scipy.stats as stats

# For each query, you can define what the expected ranking is.
EXPECTED_RANKING = {
    "This is a query document": ["id1", "id3", "id2"],
    "This is a another query document": ["id2", "id3", "id1"],
}


def measure_correlation(row: "pandas.core.series.Series", ranking_column_name: str = "top doc ids") -> float:
    r"""
    A simple test that compares the expected ranking for a given query with the actual ranking produced
    by the embedding function being tested.
    """
    input_query = row["query_texts"]
    correlation, _ = stats.spearmanr(row[ranking_column_name], EXPECTED_RANKING[input_query])
    return correlation

Finally, we can evaluate and visualize the results.

In [8]:
experiment.evaluate("ranking_correlation", measure_correlation)

In [9]:
experiment.visualize()

,query_texts,embed_fn,top doc ids,distances,documents,latency,ranking_correlation
0,This is a query document,paraphrase-MiniLM-L3-v2,"[id1, id3, id2]","[14.106966018676758, 14.294026374816895, 18.137874603271484]","[This is a document, This is the document., This is another document]",0.008508,1.0
1,This is a another query document,paraphrase-MiniLM-L3-v2,"[id2, id3, id1]","[13.375584602355957, 16.815608978271484, 16.913410186767578]","[This is another document, This is the document., This is a document]",0.006097,1.0
2,This is a query document,default,"[id1, id3, id2]","[0.7111212611198425, 0.8084275126457214, 1.010977029800415]","[This is a document, This is the document., This is another document]",0.020234,1.0
3,This is a another query document,default,"[id2, id1, id3]","[0.7673601508140564, 0.8709302544593811, 0.9072309732437134]","[This is another document, This is a document, This is the document.]",0.020889,-1.0


You can also import the built-in version of the rank correlation function.

In [10]:
from prompttools.utils import ranking_correlation

EXPECTED_RANKING_LIST = [
    ["id1", "id3", "id2"],
    ["id2", "id3", "id1"],
    ["id1", "id3", "id2"],
    ["id2", "id3", "id1"],
]

experiment.run()
experiment.evaluate("ranking_correlation", ranking_correlation, expected_ranking=EXPECTED_RANKING_LIST)
experiment.visualize()

/Users/kevin/miniconda3/envs/prompttools/lib/python3.11/site-packages/chromadb/utils/read_write_lock.py:29: DeprecationWarning: notifyAll() is deprecated, use notify_all() instead
  self._read_ready.notifyAll()


,query_texts,embed_fn,top doc ids,distances,documents,latency,ranking_correlation
0,This is a query document,paraphrase-MiniLM-L3-v2,"[id1, id3, id2]","[14.106966018676758, 14.294026374816895, 18.137874603271484]","[This is a document, This is the document., This is another document]",0.005949,1.0
1,This is a another query document,paraphrase-MiniLM-L3-v2,"[id2, id3, id1]","[13.375584602355957, 16.815608978271484, 16.913410186767578]","[This is another document, This is the document., This is a document]",0.006486,1.0
2,This is a query document,default,"[id1, id3, id2]","[0.7111212611198425, 0.8084275126457214, 1.010977029800415]","[This is a document, This is the document., This is another document]",0.020484,1.0
3,This is a another query document,default,"[id2, id1, id3]","[0.7673601508140564, 0.8709302544593811, 0.9072309732437134]","[This is another document, This is a document, This is the document.]",0.021018,-1.0


You can also use auto evaluation, which is shown in the RAG notebook example.

## Chunking and Pre-Processing

You may want to test various chunking and pre-processsing strategies prior to storing your documents into your vector database. One example of doing that is below.

We have a built-in `chunk_text` function that allows you to chunk a source documents into chunks of varying maximum length.

After we generate the desired documents, we can set up the rest of the experiments in the same way as above. We will then run, evalaute, aggregate, and save the results.

We want to compute an aggregate score for each chunking strategy, this will allow us to compared different chunking/pre-processing strategies and choose the best one.

In [ ]:
from prompttools.utils import chunk_text

source_document = "The United States of America (U.S.A. or USA), commonly known as the United States (U.S. or US) or simply America, is a country primarily located in North America and consisting of 50 states, a federal district, five major unincorporated territories, and nine Minor Outlying Islands.[i] It includes 326 Indian reservations. It is the world's third-largest country by both land and total area.[c] It shares land borders with Canada to its north and with Mexico to its south and has maritime borders with the Bahamas, Cuba, Russia, and other nations.[j] With a population of over 333 million,[k] it is the most populous country in the Americas and the third-most populous in the world. The national capital of the United States is Washington, D.C., and its most populous city and principal financial center is New York City."

for max_chunk_len in [20, 30, 40]:  # Test different chunking strategy
    # Create documents for each strategy
    doc_chunks = chunk_text(source_document, max_chunk_len)
    n_doc = len(doc_chunks)
    metadatas = [{"source": "my_source"}] * n_doc
    ids = ["id" + str(i) for i in range(n_doc)]
    # Build the relevant parameters to write into ChromaDB
    add_to_collection_params = {
        "documents": doc_chunks,
        "metadatas": metadatas,
        "ids": ids,
    }
    # Next steps: as shown in the previous code and other notebook examples
    # 1. Set up the rest of your experiment
    # 2. Run Experiment
    # 3. Evaluate the results from this chunking strate
    # 4. Aggregate and save the results

You can further integrate this vector DB experiment with the RetrievalAugmentedGeneration (RAG) experiment. Please see the Retrieval Augmented Generation notebook example.